# **Importamos las dependencias**

In [1]:
# Instalación de todas las dependencias necesarias
%pip install google-cloud-storage google-cloud-aiplatform vertexai matplotlib seaborn scikit-learn pandas seaborn numpy python-dotenv --quiet
%pip install python-dotenv --quiet
%pip install pillow --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# --- Para el proceso de datos y visualización ---
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import time


# --- Para manejar las variables de entorno ---
import os
from dotenv import load_dotenv


# --- Desactiva las advertencias de asignaciones encadenadas en pandas para evitar mensajes de warning al modificar DataFrames.
pd.options.mode.chained_assignment = None  # default='warn'


# --- Dependencias de Vertex AI ---
import vertexai                                              # Importa el módulo principal de Vertex AI.
from vertexai import init                                    # Inicializa Vertex AI con las credenciales y configuraciones necesarias.
from vertexai.vision_models import Image as VMImage          # Importa la clase Image de Vertex AI para manejar imágenes.
from vertexai.vision_models import MultiModalEmbeddingModel  # Importa el modelo de embeddings multimodales de Vertex AI para procesar imágenes y videos.
from vertexai.vision_models import Video                     # Clase para manejar archivos de video en Vertex AI.
from vertexai.vision_models import VideoSegmentConfig        # Configuración para segmentar videos al gener


# --- Para conectarse y consultar un endpoint de búsqueda vectorial (Vector Search) en Vertex AI. 
from google.cloud.aiplatform.matching_engine import MatchingEngineIndexEndpoint 

# --- Para acceder a los buckets de Google Cloud Storage y manejar archivos.
from google.cloud import storage


# --- Dependencias para poder visualizar ---
from IPython.display import Video as MVideo                  # Permite mostrar videos directamente en celdas de Jupyter Notebook.
from IPython.display import HTML                             # Permite mostrar contenido HTML en celdas de Jupyter Notebook.
from IPython.display import Image as ImageByte               # Permite mostrar imágenes en el notebook (renombrado como ImageByte para evitar conflictos de nombres).
from IPython.display import display                          # Función general para mostrar objetos en el notebook.
from sklearn.metrics.pairwise import cosine_similarity       # Función para calcular la similitud coseno entre vectores, útil para comparar embeddings.

# **Configuración del entorno de Vertex**

In [3]:
# --- Carga las variables de entorno desde un archivo .env ---
load_dotenv()

# --- Obtenemos las variables necesarias del entorno ---

PROJECT_ID = os.getenv("PROJECT_ID")                        # ID del proyecto de Google Cloud
LOCATION = os.getenv("LOCATION")                            # Región donde se encuentran los recursos de Vertex AI
INDEX_ID = os.getenv("INDEX_ID")      # Nombre completo del endpoint de búsqueda vectorial
ENDPOINT_ID = os.getenv("ENDPOINT_ID")          # ID del índice desplegado en el endpoint
BUCKET_NAME = os.getenv("BUCKET_NAME")                      # Nombre del bucket de Google Cloud Storage
DESTINATION_BLOB_NAME = os.getenv("DESTINATION_BLOB_NAME")  # Ruta y nombre del archivo destino en el bucket

# Imprime las variables cargadas para verificación (de si lo hicimos bien todo)
print("PROJECT_ID:", PROJECT_ID)
print("INDEX_ID:", INDEX_ID)
print("ENDPOINT_ID:", ENDPOINT_ID)
print("BUCKET_NAME:", BUCKET_NAME)
print("DESTINATION_BLOB_NAME:", DESTINATION_BLOB_NAME)

PROJECT_ID: constant-setup-463820-p6
INDEX_ID: projects/640283206292/locations/us-central1/indexEndpoints/1468492336894836736
ENDPOINT_ID: mexicocostarica_1750850443317
BUCKET_NAME: vboxiooof
DESTINATION_BLOB_NAME: Embeddings


In [3]:
# --- Inicializamos Vertex AI ---
init(project = PROJECT_ID, location = LOCATION)

# **Funciones**

In [ ]:
# --- Función para generar embeddings de videos ---

def get_video_embedding(ruta_video: str) -> list: 
    
    """
    Genera un embedding para un video dado.

    Args:
        ruta_video (str): Ruta al archivo de video.

    Returns:
        list: Embedding del video.
    """
    # Cargamos el video desde la ruta proporcionada
    video = Video.load_from_file(ruta_video)
    
    # Genera el embedding del video utilizando el modelo de embeddings multimodales
    embedding = mm_embendding_model.get_embeddings(video = video, 
                                                   video_segment_config = VideoSegmentConfig(interval_sec=4) # Configura el segmento del video para generar embeddings cada 4 segundos.
                                                  )
    
    return [video_emb.embedding for video_emb in embedding.video_embeddings]  # Retorna una lista de embeddings para cada segmento del video.



# --- Función para generar embeddings de texto ---

def get_text_embedding(text: str) -> list:

    print(f"Generando embedding para el texto: '{text}'")
    embeddings = mm_embendding_model.get_embeddings(
        contextual_text=text,
    )

    return embeddings.text_embedding



# --- Función para generar embeddings de imágenes ---

def get_image_embedding(
    image_path: str,
    dimension: int | None = 1408,
) -> list[float]:
    image = VMImage.load_from_file(image_path)
    embedding = mm_embendding_model.get_embeddings(
        image=image,
        dimension=dimension,
    )
    return embedding.image_embedding



# --- Función para buscar vecinos más cercanos en el índice de búsqueda vectorial --- 

def find_nearest_neighbors(query_embedding: list, num_neighbors: int = 20): # Cambia el número de vecinos a buscar según tus necesidades.

    print("Conectando al Index Endpoint...")
    index_endpoint = MatchingEngineIndexEndpoint(index_endpoint_name=INDEX_ID)
    
    print(f"Buscando los {num_neighbors} videos más similares...")
    neighbors = index_endpoint.find_neighbors(

        deployed_index_id=ENDPOINT_ID,
        queries=[query_embedding],
        num_neighbors=num_neighbors

    )

    return neighbors



# --- Función para mostrar un segmento de video específico en el notebook ---

def display_video_segment(video_gcs_uri: str, segment_id: str, interval: int):

    try:
        # Extraemos el número del segmento del ID. Ej: "VIDEOYAGO_segment_5" -> 5
        segment_number = int(segment_id.split('_')[-1])
        start_time = segment_number * interval # El intervalo (al menos en este notebook) es de 4 segundos.
        end_time = start_time + interval
        
        # Convertimos la URI de gs:// a una URL pública de https://
        public_url = video_gcs_uri.replace("gs://", "https://storage.googleapis.com/")
        
        # Creamos el código HTML para el video, apuntando al tiempo de inicio
        video_html = f"""
        <p>Mostrando segmento: <b>{segment_id}</b> (segundos {start_time}-{end_time})</p>
        <video width="640" controls>
            <source src="{public_url}#t={start_time},{end_time}" type="video/mp4">
            Tu navegador no soporta la etiqueta de video.
        </video>
        """
        display(HTML(video_html))
        
    except (ValueError, IndexError) as e:
        print(f"No se pudo parsear el ID del segmento '{segment_id}'. Error: {e}")




# --- Convierte una URI de Google Cloud Storage a una URL pública accesible por HTTP ---

def get_public_url_from_gcs(gcs_uri: str) -> str:
    """
    Convierte una URI de Google Cloud Storage (gs://bucket/archivo) a una URL pública HTTP.

    Args:
        gcs_uri (str): URI de Google Cloud Storage.

    Returns:
        str: URL pública accesible desde el navegador.
    """
    return gcs_uri.replace("gs://", "https://storage.googleapis.com/").replace(
        " ", "%20"
    )



# --- Muestra un video almacenado en Google Cloud Storage en el notebook ---

def display_video_from_gcs(gcs_uri: str) -> None:
    """
    Muestra un video almacenado en Google Cloud Storage directamente en el notebook.

    Args:
        gcs_uri (str): URI de Google Cloud Storage del video.
    """
    public_url = get_public_url_from_gcs(gcs_uri)
    display(
        HTML(
            f"""
            <video width="480" controls>
                <source src="{public_url}" type="video/mp4">
                Tu navegador no soporta la reproducción de video.
            </video>
            """
        )
    )


# --- Función para imprimir videos similares basados en embeddings ---

def print_similar_videos(query_emb: list[float], data_frame: pd.DataFrame):
    """
    Calcula la similitud (producto punto) entre un embedding de consulta y los embeddings de videos almacenados en un DataFrame.
    Muestra los videos más similares y despliega el video más relevante en el notebook.

    Args:
        query_emb (list[float]): Embedding de consulta (por ejemplo, generado a partir de un video o texto).
        data_frame (pd.DataFrame): DataFrame que contiene al menos las columnas 'video_embeddings', 'file_name' y 'gcs_path'.

    Funcionamiento:
        - Calcula el producto punto entre el embedding de consulta y cada embedding de video en el DataFrame.
        - Añade una columna 'score' con los resultados de similitud.
        - Ordena el DataFrame por 'score' de mayor a menor.
        - Imprime los nombres de los archivos y sus scores más altos.
        - Muestra el video más similar directamente en el notebook.
    """
    # Obtiene la columna de embeddings de video
    video_embs = data_frame["video_embeddings"]

    # Calcula el producto punto entre cada embedding y el de consulta
    scores = [np.dot(eval(video_emb), query_emb) for video_emb in video_embs]
    data_frame["score"] = scores

    # Ordena por score descendente
    data_frame = data_frame.sort_values(by="score", ascending=False)

    # Imprime los resultados principales
    print(data_frame.head()[["score", "file_name"]])

    # Obtiene la URL GCS del video más similar
    url = data_frame.iloc[0]["gcs_path"]

    # Muestra el video en el notebook
    display_video_from_gcs(url)



# --- Función para guardar embeddings en Google Cloud Storage como JSONL ---

def guardar_embeddings_en_gcs(
    project_id: str,
    bucket_name: str,
    blob_name: str,
    ids: list[str],
    embeddings: list[list[float]]
):
    """
    Convierte una lista de IDs y embeddings al formato JSONL y lo sube a GCS.

    Args:
        project_id (str): Tu proyecto de Google Cloud.
        bucket_name (str): El nombre del bucket de destino.
        blob_name (str): La ruta y nombre del archivo a crear en el bucket.
        ids (list[str]): Lista de IDs únicos para cada embedding.
        embeddings (list[list[float]]): La lista de vectores de embedding.
    """
    print(f"Conectando al bucket '{bucket_name}'...")
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    print(f"Escribiendo {len(ids)} embeddings en el archivo en memoria...")
    
    # Usamos un context manager para escribir directamente al archivo en GCS
    with blob.open("w") as f:
        for i, embedding in zip(ids, embeddings):
            # Creamos el diccionario para la línea actual
            data_point = {"id": i, "embedding": embedding}
            # Lo convertimos a un string JSON y escribimos la línea en el archivo
            f.write(json.dumps(data_point) + "\n")

    print(f"¡Éxito! Archivo '{blob_name}' subido correctamente a 'gs://{bucket_name}/{blob_name}'.")



    # --- Vemos el contenido de nuestro bucket de GCS ---
    
def list_blobs(bucket_name: str):
    """
    Guarda en una lista los blobs (archivos) en un bucket de Google Cloud Storage.

    Args:
        bucket_name (str): El nombre del bucket de GCS.
    """
    ls = []
    storage_client = storage.Client()
    blobs = storage_client.list_blobs(bucket_name)

    print(f"Archivos en el bucket '{bucket_name}':")
    for blob in blobs:
        #print(blob.name)
        ls.append(blob.name)
    
    return ls[4:]  # Retorna la lista de blobs, omitiendo los primeros 4 elementos
    



In [7]:
get_public_url_from_gcs('gs://vboxioof/Videos/Videos_Segmentados/clip_1.mp4')

'https://storage.googleapis.com/vboxioof/Videos/Videos_Segmentados/clip_1.mp4'

# **Generamos los embeddings**

In [7]:
# --- Cargamos el modelo de embeddings multimodales ---
mm_embendding_model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

In [29]:
# ruta_video = "gs://vboxioof/Videos/Videos Largos/azteca7-2025-05-23.mkv" 

# Obtenemos todas los nombres de los clips del video en el bucket
clips = list_blobs(BUCKET_NAME)  # Llama a la función para listar los blobs en el bucket especificado.

lsEmbeddings = []  # Lista para almacenar los embeddings de los videos


# Marcamos el inicio del proceso de generación de embeddings
start_time = time.time()

# Generamos el embedding del video
for clip in clips[1:]: # Omitimos el primer elemento de la lista, que es un directorio (y no un archivo).
    print(f"Embedding del video: {os.path.basename(clip)}")
    video_embedding = get_video_embedding("gs://vboxiooof/"+clip) # -> Ojo aquí, le agruegé una 'o' extra al bucket, porque si no, no me funcionaba
    lsEmbeddings.append(video_embedding)  # Añade el embedding a la lista
    
end_time = time.time()  # Marcamos el final del proceso de generación de embeddings

print(f"Tiempo total para generar embeddings: {end_time - start_time:.4f} segundos")  # Imprime el tiempo total transcurrido

Archivos en el bucket 'vboxiooof':
Embedding del video: mexicosta_segment_001.mkv
Embedding del video: mexicosta_segment_002.mkv
Embedding del video: mexicosta_segment_003.mkv
Embedding del video: mexicosta_segment_004.mkv
Embedding del video: mexicosta_segment_005.mkv
Embedding del video: mexicosta_segment_006.mkv
Embedding del video: mexicosta_segment_007.mkv
Embedding del video: mexicosta_segment_008.mkv
Embedding del video: mexicosta_segment_009.mkv
Embedding del video: mexicosta_segment_010.mkv
Embedding del video: mexicosta_segment_011.mkv
Embedding del video: mexicosta_segment_012.mkv
Embedding del video: mexicosta_segment_013.mkv
Embedding del video: mexicosta_segment_014.mkv
Embedding del video: mexicosta_segment_015.mkv
Embedding del video: mexicosta_segment_016.mkv
Embedding del video: mexicosta_segment_017.mkv
Embedding del video: mexicosta_segment_018.mkv
Embedding del video: mexicosta_segment_019.mkv
Embedding del video: mexicosta_segment_020.mkv
Embedding del video: mexi

In [30]:
# Vemos cuantos embeddings se generaron
print(f"Cantidad de elementos en la lista de embeddings: {len(lsEmbeddings)}")

Cantidad de elementos en la lista de embeddings: 483


In [32]:
# Vemos la cantidad de embeddings generados
sum = 0
for em in lsEmbeddings:
    sum += len(em)  # Suma la longitud de cada embedding (debería ser 1408)

print(f"Cantidad total de embeddings: {sum}")  # Imprime la suma total de elementos en los embeddings

Cantidad total de embeddings: 1931


# **Carga a Vector Search**

In [33]:
# --- Hacemos un 'aplanamiento' de la lista de embeddings para que sea un vector 1D ---
FlatListEmbeddings = [embedding for sublist in lsEmbeddings for embedding in sublist]  # Aplana la lista de listas en una sola lista de embeddings

print(f"Cantidad de embeddings a guardar: {len(FlatListEmbeddings)}")

Cantidad de embeddings a guardar: 1931


In [34]:
# --- GENERACIÓN DE IDs ---
ids_de_embeddings = [f"mexicosta_segment_{i}" for i in range(len(FlatListEmbeddings))]


if len(ids_de_embeddings) != len(FlatListEmbeddings):
        raise ValueError("La cantidad de IDs no coincide con la cantidad de embeddings.")
        
guardar_embeddings_en_gcs(
    project_id=PROJECT_ID,
    bucket_name=BUCKET_NAME,
    blob_name=DESTINATION_BLOB_NAME+'/Embeddings.json',  # Asegúrate de que el nombre del archivo sea correcto
    ids=ids_de_embeddings,
    embeddings=FlatListEmbeddings
)



Conectando al bucket 'vboxiooof'...
Escribiendo 1931 embeddings en el archivo en memoria...
¡Éxito! Archivo 'Embeddings/Embeddings.json' subido correctamente a 'gs://vboxiooof/Embeddings/Embeddings.json'.


# **Query a los embeddings**

In [5]:
GCS_VIDEO_URI = "vboxioof/Videos/Videos_Segmentados/"   # La ruta a la carpeta de los clips
SEGMENT_INTERVAL_SEC = 4                                # El intervalo en segundos que usamos para segmentar el video
image_path = "gs://vboxioof/Imagenes/ATT.png"        # Ruta a una imagen de ejemplo para generar un embedding

### **Consulta con imagenes**

In [ ]:
if __name__ == "__main__":

    # --- Cargamos la imagen, y le hacemos un embedding ---
    image_embedding = get_image_embedding(image_path=image_path)


    # --- Buscamos los vecinos más cercanos al embedding de la imagen ---
    search_results = find_nearest_neighbors(image_embedding, 50)


    # Mostramos los resultados
    print("\n--- RESULTADOS DE LA BÚSQUEDA ---")
    
    if not search_results or not search_results[0]:
        print("No se encontraron resultados.")
        
    else:

        neighbors_sorted = sorted(search_results[0], key=lambda x: x.distance)

        for neighbor in neighbors_sorted:
            
            video_segment_id = neighbor.id
            distancia = neighbor.distance

            # if distancia < 0.1:  # Si la distancia es mayor a 0.1, no lo consideramos relevante

            print(f"\nEncontrado: [ID: {video_segment_id}] - [Distancia: {distancia:.8f}]")
            
            # Mostramos el fragmento de video correspondiente
            display_video_segment(
                video_gcs_uri=GCS_VIDEO_URI +f"{video_segment_id}.mp4",
                segment_id=video_segment_id,
                interval=SEGMENT_INTERVAL_SEC
            )

            # else:
                # print(f"El segmento {video_segment_id} no es relevante (distancia: {distancia:.8f}).")

        print("\n--- FIN DE LOS RESULTADOS ---")



Conectando al Index Endpoint...
Buscando los 50 videos más similares...

--- RESULTADOS DE LA BÚSQUEDA ---

Encontrado: [ID: PartidoAzteca7_segment_826] - [Distancia: 0.47895408]



Encontrado: [ID: PartidoAzteca7_segment_932] - [Distancia: 0.47994792]



Encontrado: [ID: PartidoAzteca7_segment_958] - [Distancia: 0.48236069]



Encontrado: [ID: PartidoAzteca7_segment_724] - [Distancia: 0.48360842]



Encontrado: [ID: PartidoAzteca7_segment_801] - [Distancia: 0.48797610]


...


Encontrado: [ID: PartidoAzteca7_segment_654] - [Distancia: 0.58067942]



Encontrado: [ID: PartidoAzteca7_segment_700] - [Distancia: 0.59421754]



Encontrado: [ID: PartidoAzteca7_segment_647] - [Distancia: 0.60155731]



Encontrado: [ID: PartidoAzteca7_segment_763] - [Distancia: 0.61714351]



--- FIN DE LOS RESULTADOS ---


### **Consulta con lenguaje natural**

In [12]:
# --- EJECUCIÓN DE LA BÚSQUEDA ---
if __name__ == '__main__':
    
    # ------------------------------------------------------------------
    # AQUÍ PONES TU CONSULTA EN LENGUAJE NATURAL
    texto_de_busqueda = "Caliente"
    # ------------------------------------------------------------------
    
    # 1. Obtenemos el embedding del texto de búsqueda
    text_query = get_text_embedding(texto_de_busqueda)
    
    # 2. Buscamos en Vector Search usando el embedding del texto
    search_results = find_nearest_neighbors(text_query, 5)
    
    # 3. Mostramos los resultados
    print("\n--- RESULTADOS DE LA BÚSQUEDA ---")
    
    if not search_results or not search_results[0]:
        print("No se encontraron resultados.")
        
    else:

        neighbors_sorted = sorted(search_results[0], key=lambda x: x.distance)

        print(f"Se encontraron {len(neighbors_sorted)} resultados.")

        for neighbor in neighbors_sorted:
            
            video_segment_id = neighbor.id
            distancia = neighbor.distance

            if distancia < 0.2:  # Si la distancia es mayor a 0.1, no lo consideramos relevante

                print(f"\nEncontrado: [ID: {video_segment_id}] - [Distancia: {distancia:.8f}]")
            
                # Mostramos el fragmento de video correspondiente
                display_video_segment(
                    video_gcs_uri=GCS_VIDEO_URI +f"{video_segment_id}.mp4",
                    segment_id=video_segment_id,
                    interval=SEGMENT_INTERVAL_SEC
                )

            else:
                pass

        print("\n--- FIN DE LOS RESULTADOS ---")

Generando embedding para el texto: 'Caliente'
Conectando al Index Endpoint...
Buscando los 5 videos más similares...

--- RESULTADOS DE LA BÚSQUEDA ---
Se encontraron 5 resultados.

Encontrado: [ID: mexicosta_segment_167] - [Distancia: 0.12898469]



Encontrado: [ID: mexicosta_segment_1095] - [Distancia: 0.13174450]



Encontrado: [ID: mexicosta_segment_905] - [Distancia: 0.13280150]



Encontrado: [ID: mexicosta_segment_162] - [Distancia: 0.14469939]



Encontrado: [ID: mexicosta_segment_1090] - [Distancia: 0.14725038]



--- FIN DE LOS RESULTADOS ---
